In [18]:
#pip install pandas langchain openai tiktoken

from langchain_core.documents import Document

In [19]:
import pandas as pd
from pathlib import Path

# 1. 입력 CSV 경로들
CSV_FILES = [
    "data/processed/law/law_labeled.csv",
    "data/processed/rule/rule_labeled.csv",
    "data/processed/case/case_labeled.csv",
]

# 2. CSV 로드
dfs = []
for path in CSV_FILES:
    df = pd.read_csv(path)
    dfs.append(df)

# 3. 하나로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 4. (선택) id 중복 체크
dup_ids = merged_df["id"].duplicated().sum()
print(f"중복 id 개수: {dup_ids}")

# 5. 저장
output_path = Path("data/processed/all_chunks.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

merged_df.to_csv(output_path, index=False)

print(f"✅ 병합 완료: {output_path}")
print(f"총 청크 수: {len(merged_df)}")

중복 id 개수: 0
✅ 병합 완료: data\processed\all_chunks.csv
총 청크 수: 248


In [20]:
import pandas as pd
from langchain_core.documents import Document

def load_chunked_csv(csv_path: str):
    df = pd.read_csv(csv_path)

    documents = []
    for _, row in df.iterrows():
        documents.append(
            Document(
                page_content=row["text"],
                metadata={
                    "id": row["id"],
                    "category": row["category"],
                    "priority": int(row["priority"]),
                    "article": row["article"],
                    "chunk_id": row["chunk_id"],
                    "source": row["source"],
                }
            )
        )
    return documents

In [21]:
docs = load_chunked_csv("data/processed/all_chunks.csv")

In [22]:
docs[0]

Document(metadata={'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}, page_content='(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차')

In [23]:
docs[0].page_content

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [24]:
docs[0].page_content[:300]

'(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.\n제2장 계약\n제6절 사용대차'

In [25]:
for i in range(5):
    print(f"\n--- doc {i} ---")
    print(docs[i].metadata)
    print(docs[i].page_content[:200])


--- doc 0 ---
{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f', 'category': 'law', 'priority': 4, 'article': '제390조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(채무불이행과 손해배상) 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다.
제2장 계약
제6절 사용대차

--- doc 1 ---
{'id': '767cb115-6be6-441b-a624-0de4fdd5d05d', 'category': 'law', 'priority': 4, 'article': '제618조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(임대차의 의의) 임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방이 이에 대하여 차임을 지급할 것을 약정함으로써 그 효력이 생긴다.

--- doc 2 ---
{'id': '1bb029e6-f430-4d8e-8777-f93e7ac57c90', 'category': 'law', 'priority': 4, 'article': '제619조', 'chunk_id': '본문', 'source': '민법(법률)(제20432호)(20260101)-간략.docx'}
(처분능력, 권한없는 자의 할 수 있는 단기임대차) 처분의 능력 또는 권한없는 자가 임대차를 하는 경우에는 그 임대차는 다음 각호의 기간을 넘지 못한다.
1. 식목, 채염 또는 석조, 석회조, 연와조 및 이와 유사한 건축을 목적으로 한 토지의 임대차는 10년
2. 기타 토지의 임대차는 5년
3. 건물 기타 공작물의 임대차는 3년
4. 동산의 임대차는 6월

--- doc 3 ---
{'id': '1527dfbd-1069-

In [26]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

texts = [doc.page_content for doc in docs]

vectors = embeddings.embed_documents(texts)

print(f"임베딩 완료: {len(vectors)}")
print(f"벡터 차원: {len(vectors[0])}")

임베딩 완료: 248
벡터 차원: 3072


In [28]:
import numpy as np

np.save(
    "data/processed/law_vectors.npy",
    np.array(vectors)
)

In [30]:
# import json

# metadata = [doc.metadata for doc in docs]

# with open(
#     "data/embeddings/law_metadata.json",
#     "w",
#     encoding="utf-8"
# ) as f:
#     json.dump(metadata, f, ensure_ascii=False, indent=2)

In [31]:
embedded_items = []

for doc, vector in zip(docs, vectors):
    embedded_items.append({
        "id": doc.metadata["id"],
        "vector": vector,
        "text": doc.page_content,
        "metadata": doc.metadata
    })

embedded_items[0]

{'id': 'ea20858b-3e6b-4fab-b297-a185d0f34d1f',
 'vector': [0.0012066885828971863,
  -0.04566808044910431,
  -0.00907521415501833,
  0.026063326746225357,
  -0.0060692280530929565,
  0.020154418423771858,
  -0.03412509337067604,
  -0.004022295121103525,
  -0.037423089146614075,
  -0.015425000339746475,
  0.02142552100121975,
  0.0011608829954639077,
  -0.016524333506822586,
  -0.015344841405749321,
  -0.007884271442890167,
  -0.02757490798830986,
  -0.020475056022405624,
  0.01671900600194931,
  -0.02108198031783104,
  0.005820160731673241,
  -0.03865983709692955,
  0.02498689852654934,
  0.01292860135436058,
  -0.0069853379391133785,
  -0.03694213181734085,
  0.008119024336338043,
  0.000721436575986445,
  0.01878025382757187,
  -0.00059046148089692,
  -0.02917810156941414,
  -0.027895547449588776,
  0.006687602493911982,
  -0.04143106937408447,
  -0.006973886862397194,
  0.006063502747565508,
  -0.02539914660155773,
  -0.009052311070263386,
  -0.019845230504870415,
  -0.00926416181027

In [32]:
pinecone_vectors = [
    {
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    }
    for item in embedded_items
]

# PINECONE


In [33]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index("realestate")  # 네 인덱스 이름

In [34]:
pinecone_vectors = []

for item in embedded_items:
    pinecone_vectors.append({
        "id": item["id"],
        "values": item["vector"],
        "metadata": item["metadata"]
    })

In [35]:
BATCH_SIZE = 100

for i in range(0, len(pinecone_vectors), BATCH_SIZE):
    batch = pinecone_vectors[i:i + BATCH_SIZE]
    index.upsert(vectors=batch)
    print(f"{i} ~ {i + len(batch)} 업로드 완료")

0 ~ 100 업로드 완료
100 ~ 200 업로드 완료
200 ~ 248 업로드 완료


In [36]:
res = index.query(
    vector=embedded_items[0]["vector"],
    top_k=5,
    include_metadata=True
)

for m in res["matches"]:
    print(m["score"], m["metadata"].get("article"))

0.999681473 제390조
0.584302902 제750조
0.493906051 제6조의2
0.458507568 사례 1
0.449554473 제617조


# 임베딩 결과 확인


In [41]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME = "realestate"   # 지금은 하드코딩 권장

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

emb = OpenAIEmbeddings(model="text-embedding-3-large")

def test_search(query, top_k=5):
    qvec = emb.embed_query(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True
    )
    return res

In [42]:
res = test_search("전세 보증금 우선변제 요건")
for m in res["matches"]:
    md = m["metadata"]
    print(m["score"], md.get("source"), md.get("article"))

0.528417587 주택임대차보호법(법률)(제21065호)(20260102).docx 제3조의2
0.490574837 주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx 제3조의2
0.461048126 주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx 제22조
0.457984924 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx 제3조의2
0.456645 상가건물 임대차보호법(법률)(제21065호)(20260102).docx 제5조
